# 📊 **Antigranular** Heart Disease Prediction Contest (ft. **Harvard/OpenDP** and **TPDP**)

🎉 Welcome to a new [Antigranular](https://antigranular.com) contest in collaboration with the [TPDP Workshop](https://tpdp.journalprivacyconfidentiality.org/2024/) and [Harvard's OpenDP Community Meeting](https://opendp.org/)!

🩺 This time, we are focusing on [heart condition detection](https://en.wikipedia.org/wiki/Cardiovascular_disease) using our new [TensorFlow Privacy](https://github.com/tensorflow/privacy) and [Opacus (PyTorch)](https://opacus.ai/) models!

🦜 Any questions? Head over to our [Discord](https://discord.com/invite/KJwApgXs4s)!



## 🏃‍♂️ Getting Started

In this section we will download the antigranular package and login




### 📦 Install Antigranular

This command installs the [Antigranular PyPI Package](https://pypi.org/project/antigranular/) on the local enviroment.


In [ ]:
!pip install antigranular

In [1]:
# Install the Antigranular package
!pip install antigranular &> /dev/null

### ✍ Login to the Enclave

Head over to [Competitions](https://www.antigranular.com/competitions) to find your `<user_id>`, `<user_secret>` and the competition's name and copy that command here.

![img](https://docs.antigranular.com/shots/comp_cell.png)

In [4]:
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Heart Disease Prediction Hackathon")

Dataset "Heart Disease Prediction Hackathon Dataset" loaded to the kernel as heart_disease_prediction_hackathon_dataset
Key Name                       Value Type     
---------------------------------------------
train_y                        PrivateDataFrame
train_x                        PrivateDataFrame
test_x                         DataFrame      

Connected to Antigranular server session id: eb6e3c69-8fba-4b72-80a2-12ab70998533, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [5]:
# import antigranular as ag
# session = ag.login(<client_id>,<client_secret>, competition = "Heart Disease Prediction Hackathon")

### 🤖 Using AG

You can now simply use ``%%ag`` to run code on an enclave! You can always head over to our [Docs](https://docs.antigranular.com/) to learn more about AG, but for now, we can define train and test variables as follows.

In [6]:
%%ag
x_train = heart_disease_prediction_hackathon_dataset["train_x"]
y_train = heart_disease_prediction_hackathon_dataset["train_y"]
x_test = heart_disease_prediction_hackathon_dataset["test_x"]

### 🕵️‍♂️ Exploring data

Exploring data in Antigranular involves spending your epsilon budget, be mindful of your usage but remember that the less epsilon you use, the less accurate your results will get!

In [7]:
%%ag
x_train.info()

+----+----------+-------------+---------------+---------+------------+
|    | Column   | numerical   | categorical   | dtype   | bounds     |
|----+----------+-------------+---------------+---------+------------|
|  0 | age      | True        | False         | int64   | (21, 86)   |
|  1 | sex      | True        | False         | int64   | (0, 1)     |
|  2 | bp       | True        | False         | int64   | (80, 215)  |
|  3 | ch       | True        | False         | int64   | (102, 597) |
|  4 | bs       | True        | False         | int64   | (67, 157)  |
|  5 | phr      | True        | False         | int64   | (62, 222)  |
+----+----------+-------------+---------------+---------+------------+



In [8]:
%%ag
y_train.info()

+----+-----------+-------------+---------------+---------+----------+
|    | Column    | numerical   | categorical   | dtype   | bounds   |
|----+-----------+-------------+---------------+---------+----------|
|  0 | condition | True        | False         | int64   | (0, 1)   |
+----+-----------+-------------+---------------+---------+----------+



In [9]:
%%ag
# We can start by exploring the data, carefully using our epsilon
describe = x_train.describe(eps=0.1)
ag_print(describe)

               age          sex  ...           bs          phr
count  7564.000000  7705.000000  ...  8700.000000  7621.000000
mean     56.853259     0.776229  ...   105.300801   163.436795
std      30.986535     0.236091  ...    16.361005     5.463358
min      21.000000     0.000000  ...    67.000000    62.000000
25%      30.895317     0.727723  ...    88.152472   132.018005
50%      68.439417     0.976560  ...    95.553524   155.329707
75%      60.054029     0.921835  ...   139.292442   156.349119
max      72.978647     0.965575  ...   132.452177   159.961542

[8 rows x 6 columns]



In [10]:
%%ag
# We can start by exploring the data, carefully using our epsilon
describe = y_train.describe(eps=0.1)
ag_print(describe)

         condition
count  7862.000000
mean      0.545760
std       0.495742
min       0.000000
25%       0.009797
50%       0.995590
75%       0.959023
max       0.997530



In [11]:
%%ag
# x_test is a public test set, so we can print it without using epsilon
ag_print(x_test)

      age  sex   bp   ch   bs  phr
0      71    1  128  326   95  117
1      61    1  153  270   98  123
2      59    1  113  236  106  181
3      69    0  109  151  109  108
4      55    0  137  235  101  150
...   ...  ...  ...  ...  ...  ...
1995   60    1  128  261  112  143
1996   50    1  143  216   94  100
1997   64    1  120  172   87  142
1998   56    1  158  294   82  144
1999   69    0  117  559  112  157

[2000 rows x 6 columns]



### 🎈 A quick solution

In this section we evaluate an editorial solution in AG using TensorFlow!

In [12]:
%%ag
import tensorflow as tf
from op_pandas import standard_scaler, PrivateDataFrame
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from op_tensorflow import PrivateKerasModel, PrivateDataLoader


# Normal keras model
seqM = Sequential([
    Dense(32, activation='relu', input_shape=(6,)),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Create DP keras model
dp_model = PrivateKerasModel(model=seqM, l2_norm_clip=1, noise_multiplier=1)

# Use a standard (non-DP) optimizer directly from keras.
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# PrivateKerasModel uses similar API as standard Keras
dp_model.compile(
	optimizer = optimizer,
	loss = 'binary_crossentropy',
	metrics = ["accuracy"]
)

/usr/local/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
/usr/local/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:58: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  distutils.version.LooseVersion(required_tensorflow_version)):



In [13]:
%%ag
x_train_scaled = standard_scaler(x_train, eps=.1)
x_train_scaled.info()

+----+----------+-------------+---------------+---------+------------------------------------------+
|    | Column   | numerical   | categorical   | dtype   | bounds                                   |
|----+----------+-------------+---------------+---------+------------------------------------------|
|  0 | age      | True        | False         | float64 | (-2.2905770842862943,                    |
|    |          |             |               |         | 2.1693172000394583)                      |
|  1 | sex      | True        | False         | float64 | (-1.3855107798413333,                    |
|    |          |             |               |         | 0.7037450772810906)                      |
|  2 | bp       | True        | False         | float64 | (-1.515011284824466, 2.154386949465972)  |
|  3 | ch       | True        | False         | float64 | (-1.4903275647373548, 3.490343423285833) |
|  4 | bs       | True        | False         | float64 | (-1.8229505640415027,            

In [14]:
%%ag
data_loader = PrivateDataLoader(feature_df=x_train_scaled , label_df=y_train, batch_size=4)

In [15]:
%%ag
dp_model.fit(x=data_loader, epochs=5, target_delta=1e-5)

Epoch 1/5

2000/2000 - 32s - loss: 0.5595 - accuracy: 0.7123 - 32s/epoch - 16ms/step

Epoch 2/5

2000/2000 - 28s - loss: 0.5298 - accuracy: 0.7297 - 28s/epoch - 14ms/step

Epoch 3/5

2000/2000 - 29s - loss: 0.5157 - accuracy: 0.7309 - 29s/epoch - 14ms/step

Epoch 4/5

2000/2000 - 29s - loss: 0.5004 - accuracy: 0.7390 - 29s/epoch - 14ms/step

Epoch 5/5

2000/2000 - 29s - loss: 0.4935 - accuracy: 0.7419 - 29s/epoch - 14ms/step



In [17]:
%%ag
x_test_scaler = standard_scaler(PrivateDataFrame(x_test), eps=.1)
y_pred = dp_model.predict(x_test_scaler, label_columns=["output"])

63/63 [==============================] - 1s 7ms/step



In [18]:
%%ag
# Note that the predictions are a float scalar
# so we scale it
def f(x: float) -> float:
  if x > 0.5:
    return 1
  else:
    return 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))

### 📝 Make your submission (Through AG)

Submit a prediction by simply typing `submit_predictions(your_prediction)` to find out how you rank on the leaderboard.

![img](https://www.antigranular.com/static/media/Step%209.8091828f3cff4324fe6d.png)


In [19]:
%%ag
result = submit_predictions(y_pred)

score: {'leaderboard': 0.706327796951987, 'logs': {'BIN_ACC': 0.7110239870032116, 'LIN_EPS': -0.004696190051224615}}



### 🎈 Another quick solution

In this section we evaluate an editorial solution in AG using Diffprivlib!

In [23]:
# %%ag
# # We can follow by importing the OP ("Oblivious Private") version of diffprivlib
# from op_diffprivlib.models import LinearRegression

In [ ]:
# %%ag
# # We can generate a quick regression using epsilon
# reg = LogisticRegression(epsilon=0.1, data_norm=3)

# # Fit it with 3 of the SNP predictors
# reg.fit(x_train[["age", "sex", "bp", "ch", "bs", "phr"]], y_train)

In [ ]:
# %%ag
# # Predict with the same features
# y_pred = reg.predict(x_test[["age", "sex", "bp", "ch", "bs", "phr"]])

# # Print the result
# ag_print(y_pred)

### 📩 Export your prediction

We take the prediciton out of AG to later send it to the leaderboards.

In [ ]:
# %%ag
# # Submit prediction
# result = submit_predictions(DataFrame(y_pred))

### 🎉 That's it!

Congrats! You made your first submission to the competition! Now it's time to keep exploring the data and to try to achieve a better score! Here are the next steps:


1.  🏫 Head over to our [Docs](https://docs.antigranular.com/) and discover Opacus (PyTorch), TensorFlow Privacy and other libraries available in Antigranular!
2. 🦜 Any questions? Head over to our [Discord](https://discord.com/invite/KJwApgXs4s)!

We hope you have fun and enjoy the competition!

Best of luck,

Antigranular Team